# Code generation CrewAI agent
---

This notebook presents a solution to build an agent using [CrewAI](https://docs.crewai.com/introduction). CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks. 

As a part of this solution, we will build a simple code generation agent that can use some content within a knowledge base and self reflection to provide executable and correct code. 

In [ ]:
# Install crew ai. For installation steps, follow the instructions here: https://docs.crewai.com/installation
!pip install 'crewai[tools]'

In [ ]:
import os
import json
import yaml
import time
import boto3
import random
import logging
from globals import *
from pathlib import Path
from litellm import completion
from botocore.exceptions import ClientError
from typing import Dict, List, Any, Optional

In [ ]:
# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
!pygmentize globals.py

In [ ]:
config = yaml.safe_load(Path(CONFIG_FILE).read_text())
logger.info(f"config=\n{json.dumps(config, indent=2)}")

In [ ]:
# fetch the current AWS region
region = config['general']['region']
# the region to be dynamically fetched
logger.info(f"Current AWS region: {region}")
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)

In [ ]:
role_name: str = config['general']['role_name']
account: str = boto3.client('sts').get_caller_identity()['Account']
role = f"arn:aws:iam::{account}:role/{role_name}"
logger.info(f"IAM role being used: {role}")

## Define the code generation task
---

In this portion of the solution we create a `gen_code` function that generates Python code for a given problem statement. This function retrieves the prompt template via a Bedrock agent, hydrates the template, and runs inference.

In [ ]:
# Sometimes the model refuses to generate content; we define a custom exception.
class NoContentGeneratedException(Exception):
    pass

# A canned failure response (if needed)
FAILED_RESPONSE = """
import sys

def main():
    input = sys.stdin.read
    data = input().split()
    print(data)

if __name__ == "__main__":
    main()
"""

# Regular expression to extract Python code blocks wrapped in ```python ... ```
REGEX_FOR_PY_CODE_EXTRACTION: str = r"```python\n(.*?)```"

def _process_task(model_name: str, formatted_prompt: str, inference_params: dict) -> str:
    """
    Runs inference for a prompt using the specified model.
    Retries (with delays and jitter) in case of errors.
    """
    max_retries: int = 10
    retry_delay: int = 60  # seconds
    logger.info(f"Running inference with prompt:\n{formatted_prompt}")
    
    for attempt in range(max_retries):
        try:
            response = completion(
                model=model_name,
                model_id=None,
                messages=[{"role": "user", "content": formatted_prompt}],
                max_tokens=inference_params["max_tokens"],
                temperature=inference_params["temperature"],
                n=inference_params["n"],
            )
            logger.info(f"Raw Response: {response}")
            
            # If the model returned no completion tokens, raise a custom exception.
            if response['usage']['completion_tokens'] == 0:
                content = response["choices"][0]["message"]["content"]
                raise NoContentGeneratedException(f"Completion tokens is 0, content={content}")
            
            return response["choices"][0]["message"]["content"]
        
        except NoContentGeneratedException as e:
            if attempt < max_retries - 1:
                this_retry_delay = retry_delay * (attempt + 1) + random.randint(1, 10)
                logger.error(f"{e}, attempt {attempt + 1}. Retrying in {this_retry_delay} seconds...")
                time.sleep(this_retry_delay)
                continue
            else:
                logger.error("Max retries exceeded for task (NoContentGeneratedException).")
                raise
                
        except RateLimitError as e:
            if attempt < max_retries - 1:
                this_retry_delay = retry_delay * (attempt + 1) + random.randint(1, 10)
                logger.error(f"{e}, attempt {attempt + 1}. Retrying in {this_retry_delay} seconds...")
                time.sleep(this_retry_delay)
                continue
            else:
                logger.error("Max retries exceeded for task (RateLimitError).")
                raise
                
        except Exception as e:
            logger.error(f"Unexpected error processing task: {str(e)}")
            raise

def hydrate_prompt(prompt_template: str, values: dict) -> str:
    """
    Renders a prompt template using Jinja2.
    """
    from jinja2 import Template
    template = Template(prompt_template)
    return template.render(values)

def gen_code(query: str, model_id: str) -> Optional[str]:
    """
    Uses Amazon Bedrock (via litellm) to generate Python code for a given problem statement.
    This function retrieves the prompt template via a Bedrock agent, hydrates the template,
    and runs inference.
    """
    # Get the prompt mapping from an environment variable
    mapping_str = os.environ.get('MODEL_ID_TO_PROMPT_ID_MAPPING')
    logger.info(f"MODEL_ID_TO_PROMPT_ID_MAPPING: {mapping_str}")
    if mapping_str is not None:
        model_id_to_prompt_id_mapping = json.loads(mapping_str)
    else:
        logger.error("MODEL_ID_TO_PROMPT_ID_MAPPING not set in environment.")
        return None
    
    prompt_id = model_id_to_prompt_id_mapping.get(model_id)
    logger.info(f"Found prompt_id={prompt_id} for model_id={model_id}")
    if prompt_id is not None:
        bedrock_agent = boto3.client(service_name="bedrock-agent", region_name=os.environ.get('AWS_REGION', REGION))
        prompt_info = bedrock_agent.get_prompt(promptIdentifier=prompt_id)
        prompt_template = prompt_info['variants'][0]['templateConfiguration']['text']['text']
        prompt = hydrate_prompt(prompt_template, {"question": query})
        
        inference_params = prompt_info['variants'][0]['inferenceConfiguration']['text']
        # Adjust key names for litellm: change maxTokens -> max_tokens
        inference_params["max_tokens"] = inference_params.pop("maxTokens")
        inference_params["n"] = 1
        logger.info(f"Running inference for model_id={model_id} with parameters: {inference_params}")
    else:
        logger.error(f"No prompt id found for model_id={model_id}")
        return None
    
    bedrock_model_id = f"bedrock/{model_id}"
    generated_text = _process_task(bedrock_model_id, prompt, inference_params)
    return generated_text


## Create a tool and assign it as a task
---

Now, we will create a code generation tool that will wrap the code generation functionality. This tool will be registered to the crewAI agent and invoked once a user asks a question.

In [ ]:
# if it is us-east-1 or us-west-2, use the inference profile model id
if "us-east-1" in region or "us-west-2" in region:
    CODE_GEN_MODEL_ID = f"us.{CODE_GEN_MODEL_ID}"
    logger.info(f"Using inference profile model id: {CODE_GEN_MODEL_ID}")

In [ ]:
import re
from crewai.tools import tool

@tool("Code Generation Tool")
def code_generation_tool(question: str) -> str:
    """
    This tool generates Python code for a given USACO problem statement.
    
    It returns a solution that is wrapped in markdown code block delimiters.
    The agent will extract the code block for further processing.
    """
    # Dummy code-generation logic for demonstration.
    generated_text = gen_code(question, model_id=CODE_GEN_MODEL_ID)
    if generated_text is None:
        return "Error: Code generation failed."
    # Attempt to extract a Python code block wrapped by ```python ... ```
    regex = r"```python\n(.*?)```"
    match = re.search(regex, generated_text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return generated_text.strip()

## Create the CrewAI agent

In [ ]:
from crewai import LLM
# customize your own LLM
amazon_nova_llm = LLM(
    model=CODE_GEN_MODEL_ID,  
    temperature=0.1,
    max_tokens=256,
    top_p=0.9,
)

In [ ]:
from crewai import Agent, Task, Crew

# Create a Code Generation Agent that registers the CodeGenerationTool.
code_gen_agent = Agent(
    role="Code Generation Agent",
    goal="Generate Python code for USACO problems.",
    backstory="An expert in code generation using Amazon Bedrock, focused on USACO challenges.",
    tools=[code_generation_tool],
    verbose=True, 
    llm=amazon_nova_llm,
)

# Define a task for code generation.
code_task = Task(
    description="Generate Python code for the following USACO problem: {question}",
    expected_output="Python code solution for the provided USACO problem.",
    agent=code_gen_agent,
    output_file="generated_code.py"
)

In [ ]:
# Crew that includes the Code Generation Agent.
crew = Crew(
    agents=[code_gen_agent],
    tasks=[code_task],
    verbose=True,
    planning=True,
    planning_llm=amazon_nova_llm
)

In [ ]:
problem_statement = ("What is the code to upload files to Amazon S3?").strip()
crew_inputs = {"question": problem_statement}

In [ ]:
# Kick off the Crew. This will delegate the task to the Code Generation Agent,
# which in turn will use the CodeGenerationTool to generate the Python code.
crew.kickoff(inputs=crew_inputs)